<a href="https://colab.research.google.com/github/Bhavya-31/9921103089/blob/main/EV_streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [15]:
# Write the Streamlit app to app.py
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Generate synthetic data for training models
num_samples = 100
temperature_data = np.random.uniform(20, 60, num_samples)
charge_cycles_data = np.random.randint(100, 1000, num_samples)
failure_labels = np.random.randint(0, 2, num_samples)  # 0 for stable, 1 for maintenance needed

# Training data for predictive maintenance model
X_maintenance = pd.DataFrame({'temperature': temperature_data, 'charge_cycles': charge_cycles_data})
y_maintenance = failure_labels
maintenance_model = GradientBoostingClassifier()
maintenance_model.fit(X_maintenance, y_maintenance)

# Synthetic data for thermal management model
ambient_temperature_data = np.random.uniform(15, 35, num_samples)
battery_usage_data = np.random.randint(100, 1000, num_samples)
battery_temp_data = 0.7 * ambient_temperature_data + 0.3 * (battery_usage_data / 50) + np.random.normal(0, 2, num_samples)

# Training data for thermal management model
X_thermal = pd.DataFrame({'ambient_temperature': ambient_temperature_data, 'battery_usage': battery_usage_data})
y_thermal = battery_temp_data
thermal_model = LinearRegression()
thermal_model.fit(X_thermal, y_thermal)

# Synthetic data for optimized charging model
distance_needed_data = np.random.uniform(10, 150, num_samples)
battery_soc_data = np.random.uniform(20, 100, num_samples)
optimal_charging_time_data = (150 - battery_soc_data) + (distance_needed_data * 0.2) + (35 - temperature_data) * 0.5 + np.random.normal(0, 5, num_samples)

# Training data for optimized charging model
X_charging = pd.DataFrame({'temperature': temperature_data, 'distance_needed': distance_needed_data, 'battery_soc': battery_soc_data})
y_charging = optimal_charging_time_data
charging_model = RandomForestRegressor()
charging_model.fit(X_charging, y_charging)

# Define the simulation functions
def predictive_maintenance(battery_temp, charge_cycles, model):
    input_data = pd.DataFrame({'temperature': [battery_temp], 'charge_cycles': [charge_cycles]})
    maintenance_prediction = model.predict(input_data)[0]
    return "Maintenance Needed" if maintenance_prediction == 1 else "Battery Stable"

def thermal_management(ambient_temp, battery_usage, thermal_model):
    input_data = pd.DataFrame({'ambient_temperature': [ambient_temp], 'battery_usage': [battery_usage]})
    predicted_temp = thermal_model.predict(input_data)[0]
    return (predicted_temp - 5, "Cooling Activated") if predicted_temp > 40 else (predicted_temp, "No Cooling Needed")

def optimized_charging(temperature, distance_needed, battery_soc, charging_model):
    input_data = pd.DataFrame({'temperature': [temperature], 'distance_needed': [distance_needed], 'battery_soc': [battery_soc]})
    return charging_model.predict(input_data)[0]

def v2g_integration(battery_level, grid_demand):
    CHARGE_THRESHOLD = 20
    GRID_ENERGY_PRICE = 0.15
    PEER_ENERGY_PRICE = 0.10
    if battery_level > CHARGE_THRESHOLD:
        energy_contributed = min((battery_level - CHARGE_THRESHOLD) / 100 * 100, grid_demand)
        revenue_from_grid = energy_contributed * GRID_ENERGY_PRICE
        energy_to_peer = (battery_level - CHARGE_THRESHOLD) / 200 * 100
        revenue_from_peer = energy_to_peer * PEER_ENERGY_PRICE
        return revenue_from_grid, revenue_from_peer, "Contributing to Grid and Peers"
    else:
        return 0, 0, "Battery below threshold; no contribution"

def ev_system_simulation(hour, ambient_temp, battery_usage, temperature, distance_needed, battery_soc, grid_demand):
    maintenance_status = predictive_maintenance(temperature, battery_usage, maintenance_model)
    battery_temp, cooling_status = thermal_management(ambient_temp, battery_usage, thermal_model)
    optimal_charging_time = optimized_charging(temperature, distance_needed, battery_soc, charging_model)
    revenue_from_grid, revenue_from_peer, v2g_status = v2g_integration(battery_soc, grid_demand)
    return {
        'hour': hour,
        'maintenance_status': maintenance_status,
        'battery_temp': battery_temp,
        'cooling_status': cooling_status,
        'optimal_charging_time': optimal_charging_time,
        'revenue_from_grid': revenue_from_grid,
        'revenue_from_peer': revenue_from_peer,
        'v2g_status': v2g_status
    }

# Streamlit Interface
st.title("EV Battery Optimization Dashboard")

st.sidebar.header("Input Parameters")
hour = 1
ambient_temp = st.sidebar.slider("Ambient Temperature (°C)", 15, 35, 25)
battery_usage = st.sidebar.slider("Battery Usage (Charge Cycles)", 100, 1000, 500)
temperature = st.sidebar.slider("Battery Temperature (°C)", 20, 60, 40)
distance_needed = st.sidebar.slider("Distance Needed (km)", 10, 150, 75)
battery_soc = st.sidebar.slider("Battery State of Charge (%)", 20, 100, 50)
grid_demand = st.sidebar.slider("Grid Demand (kWh)", 50, 150, 100)

result = ev_system_simulation(
    hour=hour,
    ambient_temp=ambient_temp,
    battery_usage=battery_usage,
    temperature=temperature,
    distance_needed=distance_needed,
    battery_soc=battery_soc,
    grid_demand=grid_demand
)

st.subheader("Simulation Results")
st.write(f"Hour: {result['hour']}")
st.write(f"Predictive Maintenance Status: {result['maintenance_status']}")
st.write(f"Battery Temperature after Thermal Management: {result['battery_temp']:.2f}°C")
st.write(f"Cooling Status: {result['cooling_status']}")
st.write(f"Optimal Charging Time: {result['optimal_charging_time']:.2f} minutes")
st.write(f"Revenue from Grid Contribution: ${result['revenue_from_grid']:.2f}")
st.write(f"Revenue from Peer-to-Peer Contribution: ${result['revenue_from_peer']:.2f}")
st.write(f"V2G Status: {result['v2g_status']}")

# Visualization
st.subheader("Revenue Visualization")
fig, ax = plt.subplots()
ax.bar(['Grid Revenue', 'Peer Revenue'], [result['revenue_from_grid'], result['revenue_from_peer']], color=['orange', 'purple'])
ax.set_ylabel("Revenue ($)")
st.pyplot(fig)
""")


In [4]:
from pyngrok import ngrok

# Replace 'YOUR_AUTH_TOKEN' with the actual authtoken you copied
!ngrok config add-authtoken 2okaxnvQwzYJCRPZYWhrmhU7xD5_7W7nCTkNENYBRSJpnEz3a


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from pyngrok import ngrok

# Start ngrok with the correct protocol and port
public_url = ngrok.connect(8501, "http")
print("Streamlit app running at:", public_url)

# Run the Streamlit app and set host to 0.0.0.0 to allow external access
!streamlit run app.py --server.address 0.0.0.0 --server.port 8501


Streamlit app running at: NgrokTunnel: "https://7be1-35-225-41-112.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501



  Stopping...
  Stopping...
